In [ ]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp midi

<IPython.core.display.Javascript object>

In [ ]:
# export
from typing import Optional, Tuple

from music21 import converter, instrument, note, chord
from music21.midi import MidiException
import pandas as pd

<IPython.core.display.Javascript object>

In [ ]:
# test
from test import test_eq
from music21 import volume, duration

<IPython.core.display.Javascript object>

# Representing MIDI notes as triplets

Although MIDI notes have a lot of information, we want to distill them into a triplet -- namely pitch, duration and velocity:

In [ ]:
# export

Pitch, Duration, Velocity = str, float, int
Triplet = Tuple[Pitch, Duration, Velocity]

<IPython.core.display.Javascript object>

First we'll create some functions to turn single notes, chords and rests into these Triplets.

In [ ]:
# export


def parse_single_note(note: note.Note) -> Triplet:
    return (str(note.pitch), float(note.duration.quarterLength), note.volume.velocity)


def parse_chord(chord: chord.Chord) -> Triplet:
    return (
        ".".join(str(n) for n in chord.normalOrder),
        float(chord.duration.quarterLength),
        chord.volume.velocity,
    )


def parse_rest(rest: note.Rest) -> Triplet:
    return ("R", float(rest.duration.quarterLength), 0)

<IPython.core.display.Javascript object>

In [ ]:
# test
quarter = duration.Duration("quarter")


def loud(n: note.Note, velocity=120) -> note.Note:
    n.volume = volume.Volume(velocity=velocity)
    return n


test_eq(("C3", 1.0, 120), parse_single_note(loud(note.Note("C3", duration=quarter))))

<IPython.core.display.Javascript object>

In [ ]:
# export
def parse_midi_file(file: str) -> (Optional[pd.DataFrame], int):
    """
    Attempts to parse a midi file into a Dataframe. Returns a Dataframe or None, together with the number of notes processed.
    """
    try:
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        notes = []
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if element.__class__ == note.Note:
                notes.append(parse_single_note(element))
            elif element.__class__ == chord.Chord:
                notes.append(parse_chord(element))
            elif element.__class__ == note.Rest:
                notes.append(parse_rest(element))

        df = pd.DataFrame.from_dict(
            {
                "pitches": [[[pitch for (pitch, duration, velocity) in notes]]],
                "durations": [[[duration for (pitch, duration, velocity) in notes]]],
                "velocities": [[[velocity for (pitch, duration, velocity) in notes]]],
            }
        )
        return (df, len(notes))
    except MidiException:
        return (None, 0)
    except IndexError:
        return (None, 0)


<IPython.core.display.Javascript object>